In [1]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import (FAISS)
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough

In [5]:
#1 load the documents
pdf_loader = PyPDFLoader("Shivam_Kumar_Thakur_resume.pdf")

In [8]:
# 2. Chunk the transcript
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(pdf_loader.load())

In [11]:
#3 .create embeddings
embadding = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vectore_store = FAISS.from_documents(
    chunks,
    embadding
)


In [31]:

retriver = vectore_store.as_retriever()
chat_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)


In [32]:
#create the prompt format
prompts = PromptTemplate(
    template="""
You are a helpful assistant.
Answer ONLY using the provided transcript context.
If the context is insufficient, just say you don't know.

{context}
Question: {question}
""",
input_variables=["context", "question"]
)

def format_docs(retrieved_docs):
    return "\n\n".join(doc.page_content for doc in retrieved_docs)

parallelChain = RunnableParallel({
    "context": retriver | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
})

mainChain = parallelChain | prompts | chat_model


In [34]:
mainChain.invoke('can you summarize my resume?')

AIMessage(content='Shivam Kumar Thakur is a Senior React Native Developer with experience in mobile app development, cross-platform development, and UI/UX. He is currently working at IProgrammer in Pune, India, where he has enhanced the Vodafone Idea Ltd (Vi) app performance by 60% and implemented offline-first architecture, secure data synchronization, and Stripe payment integration.\n\nPreviously, he was a Team Lead and React Native Developer at Mobilotte Technology, leading 15+ developers and managing project lifecycles for 8+ cross-platform apps, including crypto wallets and social networking platforms. He also worked as a React Native and Backend Developer at CRTD Technologies (Wyreflow), building cross-platform mobile apps and developing RESTful APIs.\n\nHis technical skills include:\n*   **Languages:** JavaScript, TypeScript, HTML5, CSS3, Java\n*   **Frameworks/Libraries:** React Native, React.js, Redux Toolkit, Node.js, Express, Jest\n*   **Databases & Tools:** Firebase, MongoD